### Quickstart: Compare runs, choose a model, and deploy it to a REST API

In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

Matplotlib is building the font cache; this may take a moment.


In [2]:
# Load the dataset
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv", sep=";")
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [7]:
# split the data into training, validation and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)

# Training dataset
train_x = train.drop(["quality"], axis=1).values
test_x = test.drop(["quality"], axis=1).values

# Test dataset
train_y = train[["quality"]].values.ravel()
test_y = test[["quality"]].values.ravel()

# Splitting the train dataset into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

infer_signature = infer_signature(train_x, train_y)

In [ ]:
# ANN model
def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
    # Define the model archoitecture
    mean = np.mean(train_x, axis=0)
    std = np.std(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input(shape=(train_x.shape[1],)),
            keras.layers.Normalization(mean=mean, variance=std),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1),
        ]
    )

    # Compile the model
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=params["learning_rate"],
            momentum=params["momentum"],
        ),
        loss="mean_squared_error",
        metrics=[keras.metrics.RootMeanSquaredError()],
    )

    # Train the model with mlflow tracking
    with mlflow.start_run(nested=True):
        model.fit(
            train_x,
            train_y,
            epochs=epochs,
            batch_size=64,
            validation_data=(valid_x, valid_y),
            verbose=0,
        )

        # Evaluate the model on the test set
        eval_results = model.evaluate(test_x, test_y, batch_size=64, verbose=0)
        eval_rmse = eval_results[1]

        # Log the parameters and metrics to mlflow
        mlflow.log_params(params)
        mlflow.log_metric("rmse", eval_rmse)

        # Log the model
        mlflow.tensorflow.log_model(
            model,
            artifact_path="model",
            signature=infer_signature,
        )

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [9]:
def objective(params):
    # Set the number of epochs for training
    epochs = 3

    # Train the model and get the evaluation results
    results = train_model(
        params,
        epochs,
        train_x,
        train_y,
        valid_x,
        valid_y,
        test_x,
        test_y,
    )

    # Return the results
    return results

In [10]:
space = {
    "learning_rate": hp.loguniform("learning_rate", np.log(1e-5), np.log(1e-1)),
    "momentum": hp.uniform("momentum", 0.0, 1.0),
}

In [12]:
mlflow.set_experiment("/wine-quality")

# Start an MLflow run to track the hyperparameter optimization process
with mlflow.start_run():
    # Use the Tree of Parzen Estimators (TPE) algorithm for hyperparameter optimization
    trials = Trials()
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best hyperparameters, corresponding loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("best_loss", best_run["loss"])
    mlflow.tensorflow.log_model(
        best_run["model"],
        artifact_path="model",
        signature=infer_signature,
    )

    # Print the best hyperparameters and loss
    print("Best hyperparameters:", best)
    print("Best loss:", best_run["loss"])

 25%|██▌       | 1/4 [00:24<01:13, 24.66s/trial, best loss: 1.5514990091323853]

Successfully registered model 'keras-wine-quality-model'.
Created version '1' of model 'keras-wine-quality-model'.


 50%|█████     | 2/4 [00:42<00:40, 20.47s/trial, best loss: 1.5514990091323853]

Registered model 'keras-wine-quality-model' already exists. Creating a new version of this model...
Created version '2' of model 'keras-wine-quality-model'.


 75%|███████▌  | 3/4 [01:00<00:19, 19.62s/trial, best loss: 1.5514990091323853]

Registered model 'keras-wine-quality-model' already exists. Creating a new version of this model...
Created version '3' of model 'keras-wine-quality-model'.


100%|██████████| 4/4 [01:18<00:00, 19.54s/trial, best loss: 1.5514990091323853]

Registered model 'keras-wine-quality-model' already exists. Creating a new version of this model...
Created version '4' of model 'keras-wine-quality-model'.



Best hyperparameters: {'learning_rate': np.float64(0.0024202559372321266), 'momentum': np.float64(0.5156891020187091)}
Best loss: 1.5514990091323853


In [13]:
# Inference with the best model

from mlflow.models import validate_serving_input

model_uri = "runs:/4560b5e7bf2346789d2febd67d030141/model"

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)


d:\Bhavadeesh\Education\MLOps-Udemy\mlflow_starter\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


array([[4.8611298],
       [6.35666  ],
       [5.941642 ],
       ...,
       [6.2155724],
       [5.6680355],
       [5.4791927]], dtype=float32)

In [15]:
# Load model as PyFuncModel
loaded_model = mlflow.pyfunc.load_model(model_uri)

# Predict on a test example
predictions = loaded_model.predict(test_x)
predictions

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[4.8611298],
       [6.35666  ],
       [5.941642 ],
       ...,
       [6.2155724],
       [5.6680355],
       [5.4791927]], dtype=float32)

In [16]:
# Register in the model registry
mlflow.register_model(model_uri=model_uri, name="wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1743604896820, current_stage='None', description=None, last_updated_timestamp=1743604896820, name='wine-quality', run_id='4560b5e7bf2346789d2febd67d030141', run_link=None, source='file:///d:/Bhavadeesh/Education/MLOps-Udemy/mlflow_starter/2-DLMLFLOW/mlruns/422444038188867706/4560b5e7bf2346789d2febd67d030141/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>